# Segmenting and Clustering Neighborhoods in Toronto

#### This Notebook corresponds to the Applied Data Science Capstone Week 3 Coursera Course Assignment.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <p><a href="#section1">Section 1:</a></p>
    <p>1.1. <a href="#item1.1">Importing all required Libraries and Dependancies</a></p>
    <p>1.2. <a href="#item1.2">Loading and Exploring Dataset</a></p>
    <p>1.3. <a href="#item1.3">Transforming the Dataframe</a></p>
<p></p>
2. <p><a href="#section2">Section 2:</a></p>
    <p>2.1. <a href="#item2.1">Importing Geografical Coordinates</a></p>
    <p>2.2. <a href="#item2.2"> Appending Coordinates Columns to the Dataframe</a></p>
<p></p>
3. <p><a href="#section3">Section 3:</a>
    <p>3.1. <a href="#item3.1">Explore Neighborhoods in Toronto City</a></p>
    <p>3.2. <a href="#item3.2">Analyze Each Neighborhood</a></p>
    <p>3.3. <a href="#item3.3">Cluster Neighborhoods</a></p>
    <p>3.4. <a href="#item3.4">Examine Clusters</a></p>    
</font>
</div>

<a id='section1'></a>
## 1. Section 1

   <a id='item1.1'></a>
   ### 1.1. Importing all the required libraries and dependencies.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # installing folium libraries
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages

<a id='item1'></a>

<a id="item1.2"></a>
### 1.2 Loading and Exploring the Dataset

In this section, Toronto data will be imported from Wikipedia using pandas pd,read_html. The firs 10 rows will be displayed to visualize the imported data.

In [2]:
#Calls all the tables from the webpage url and store them as dataframes in an array, each element is a dataframe
wiki_tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0) 
#print(len(wiki_tables)) #Number of dataframes in the array
df_Toronto_raw = wiki_tables[0]
df_Toronto_raw.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


<a id="item1.3"></a>
### 1.3 Transforming the Dataframe

Now the imported raw data will be edited. A new dataframe which keeps the rows where 'Borough' column values is different than 'Not assigned' will be generated. Then, the cells column 'Neighbourhood' with values 'Not assigned' will be replaced by their correspondance value of the column 'Borough'.

In [3]:
df_Toronto_edited = df_Toronto_raw.loc[df_Toronto_raw['Borough']!='Not assigned'].copy() #define a new df where Borough is different than Not assigned
df_Toronto_edited['Neighbourhood'].replace('Not assigned', df_Toronto_edited['Borough'], inplace=True)
df_Toronto_edited.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
#

A temporary working dataframe is created and its index column is replaced with new consecutive values.

In [4]:
df_temp = df_Toronto_edited[df_Toronto_edited.duplicated(['Postcode', 'Borough'], keep=False)].copy()
#
index_df = np.linspace(start = 0, stop = len(df_temp['Borough'])-1, num = len(df_temp['Borough']))
index_df = pd.array(index_df, dtype=pd.Int64Dtype())
#
df_temp['']=index_df
df_temp.set_index('', inplace=True)
#

The next step is to find the rows with duplicated 'Postcode' values and then combine the Neighhoods in one row with the neighborhoods separated with a comma. The result is the first Table shown in the Assignment. 

In [5]:

#
for i in (df_temp.index):
    if i<len(df_temp.index)-1:
        #print (i+1, df_temp.loc[i+1]['Postcode'])
        if df_temp.loc[i+1]['Postcode'] == df_temp.loc[i]['Postcode']:
            df_temp.loc[i+1]['Neighborhood'] = df_temp.loc[i]['Neighborhood'] + ", " + df_temp.loc[i+1]['Neighborhood']
#
#
df_temp.drop_duplicates(['Postcode'], keep='last', inplace=True)
df_temp.set_index(np.arange(len(df_temp['Postcode'])), inplace=True)
#
df_temp

,Postcode,Borough,Neighborhood
0,M5A,Downtown Toronto,"Harbourfront, Regent Park"
1,M6A,North York,"Lawrence Heights, Lawrence Manor"
2,M1B,Scarborough,"Rouge, Malvern"
3,M4B,East York,"Woodbine Gardens, Parkview Hill"
4,M5B,Downtown Toronto,"Ryerson, Garden District"
5,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."
6,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
7,M3C,North York,"Flemingdon Park, Don Mills South"
8,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
9,M1E,Scarborough,"Guildwood, Morningside, West Hill"


In [6]:
print('The number of rows in the dataframe is:',df_temp.shape[0])

The number of rows in the dataframe is: 57


<a id='section2'></a>
## 2. Section 2:

<a id="item2.1"></a>
### 2.1. Importing Geografical Coordinates


Next, let's load the geografical coordinates (csv file) from the url.

In [7]:
url='https://cocl.us/Geospatial_data'
df_geocodes=pd.read_csv(url,header=0)
df_geocodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<a id="item2.2"></a>
### 2.2 Appending Coordinates Columns to the Dataframe
Appending Longitude and Latitude into a copy of the dataframe.

In [8]:
df_Toronto = df_temp.copy()
df_Toronto['Longitude']=''
df_Toronto['Latitude']=''
#
for i in (df_Toronto.index):
    for j in (df_geocodes.index):
        #print (i+1, df_temp.loc[i+1]['Postcode'])
        if df_Toronto.loc[i]['Postcode'] == df_geocodes.loc[j]['Postal Code']:
            df_Toronto.loc[i]['Longitude'] = df_geocodes.loc[j]['Longitude']
            df_Toronto.loc[i]['Latitude'] = df_geocodes.loc[j]['Latitude']
df_Toronto
#
#

,Postcode,Borough,Neighborhood,Longitude,Latitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",-79.3606,43.6543
1,M6A,North York,"Lawrence Heights, Lawrence Manor",-79.4648,43.7185
2,M1B,Scarborough,"Rouge, Malvern",-79.1944,43.8067
3,M4B,East York,"Woodbine Gardens, Parkview Hill",-79.3099,43.7064
4,M5B,Downtown Toronto,"Ryerson, Garden District",-79.3789,43.6572
5,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",-79.5547,43.6509
6,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",-79.1605,43.7845
7,M3C,North York,"Flemingdon Park, Don Mills South",-79.3409,43.7259
8,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",-79.5772,43.6435
9,M1E,Scarborough,"Guildwood, Morningside, West Hill",-79.1887,43.7636


<a id='section2'></a>
## 3. Section 3:

#### Use geopy library to get the latitude and longitude values of Toronto City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>tor_explorer</em>, as shown below.

In [10]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to use the Foursquare API to explore the neighborhoods and segment them.

You will need to define the following variables in a Python Code Cell

<p>CLIENT_ID = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare ID </p>
<p>CLIENT_SECRET = 'XXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare Secret </p>
<p>VERSION = 'XXXXXXX' # Foursquare API version </p>


In [12]:
# @remove_cell
# the above line hides the content of this cell
#
CLIENT_ID = 'XN5T0LRS2W2KA5WHSCJZPLYMP42BSLO3W3AHP1UNN3ZGTNRC' # your Foursquare ID
CLIENT_SECRET = 'RPI2LCPILRMLRJ32QFSWFE4L5S0AHXVTJ4NSV5L13DKKI1MC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [13]:
df_Toronto.loc[0]['Neighborhood']

'Harbourfront, Regent Park'

Get the neighborhood's latitude and longitude values.

In [14]:
neighborhood_latitude = df_Toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_Toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_Toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront, Regent Park are 43.6542599, -79.3606359.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [15]:
# type your answer here
latitude= 43.6542599
longitude= -79.3606359
radius= 500
LIMIT= 100

url = 'https://api.foursquare.com/v2/venues/explore?'+'client_id='+str(CLIENT_ID)+'&client_secret='+str(CLIENT_SECRET)+'&ll='+str(latitude)+','+str(longitude)+'&v='+str(VERSION)+'&radius='+str(radius)+'&limit='+str(LIMIT)
url
# send GET request and get top 100 venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d2e19bee97dfb002c2df1c2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 49,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

Send the GET request and examine the resutls

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d2e19c0dbde111dc696642b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 49,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

All the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


And how many venues were returned by Foursquare?

In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

49 venues were returned by Foursquare.


<a id='item2'></a>

<a id="item3.1"></a>
### 3.1. Explore Neighborhoods in Toronto

Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [21]:
# type your answer here

toronto_venues = getNearbyVenues(names=df_Toronto['Neighborhood'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )



Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Rouge, Malvern
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
Bathurst Manor, Downsview North, Wilson Heights
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Fairview, Henry Farm, Oriole
Northwood Park, York University
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
CFB Toronto, Downsview East
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
Clairlea, Golden Mile, Oakridge
Silver Hills, York Mills
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Downsview, North Park, Upwood Park
Cliffcrest, Cliffside, Scarborough Village West
New

#### Let's check the size of the resulting dataframe

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(1429, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Let's check how many venues were returned for each neighborhood

In [23]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",9,9,9,9,9,9
"Brockton, Exhibition Place, Parkdale Village",24,24,24,24,24,24


#### Let's find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


<a id='item3'></a>

<a id="item3.2"></a>
### 3.2. Analyze Each Neighborhood

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [26]:
toronto_onehot.shape

(1429, 236)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,0.00,0.010000,0.01,0.000000,0.030000,0.000000,0.000000,0.00,0.0000,0.030000,0.000000,0.040000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.00000,0.00,0.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.0,0.050000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.00000,0.070000,0.000000,0.000000,0.00,0.00,0.01,0.000000,0.00,0.02,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.01,0.010000,0.010000,0.000000,0.000000,0.00

#### Let's confirm the new size

In [28]:
toronto_grouped.shape

(56, 236)

#### Let's print each neighborhood along with the top 5 most common venues

In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
            venue  freq
0      Playground  0.33
1             Gym  0.33
2            Park  0.33
3   Movie Theater  0.00
4  Massage Studio  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.22
1             Pharmacy  0.11
2          Coffee Shop  0.11
3  Fried Chicken Joint  0.11
4           Beer Store  0.11


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.22
1     Coffee Shop  0.11
2    Skating Rink  0.11
3  Sandwich Place  0.11
4             Pub  0.11


----Bathurst Manor, Downsview North, Wilson Heights----
                       venue  freq
0                Coffee Shop  0.11
1      

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Bar,Asian Restaurant,Bakery,Hotel,Gym
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Gym,Playground,Park,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar
2,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Pharmacy,Sandwich Place,Beer Store,Fast Food Restaurant,Coffee Shop,Gastropub,Creperie
3,"Alderwood, Long Branch",Pizza Place,Pharmacy,Skating Rink,Pool,Pub,Sandwich Place,Coffee Shop,Gym,Gastropub,Cosmetics Shop
4,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Middle Eastern Restaurant,Sushi Restaurant,Diner,Bridal Shop,Sandwich Place,Deli / Bodega,Fast Food Restaurant,Bank,Restaurant


<a id='item4'></a>

<a id="item3.3"></a>
### 3.3. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [32]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [33]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Bar,Asian Restaurant,Bakery,Hotel,Gym
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Gym,Playground,Park,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar
2,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Pharmacy,Sandwich Place,Beer Store,Fast Food Restaurant,Coffee Shop,Gastropub,Creperie
3,"Alderwood, Long Branch",Pizza Place,Pharmacy,Skating Rink,Pool,Pub,Sandwich Place,Coffee Shop,Gym,Gastropub,Cosmetics Shop
4,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Middle Eastern Restaurant,Sushi Restaurant,Diner,Bridal Shop,Sandwich Place,Deli / Bodega,Fast Food Restaurant,Bank,Restaurant
5,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Indian Restaurant,Thai Restaurant,Butcher,Restaurant,Pub,Sushi Restaurant,Juice Bar,Café
6,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Women's Store,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
7,"Bloordale Gardens, Eringate, Markland Wood, Ol...",Pharmacy,Beer Store,Pizza Place,Convenience Store,Café,Shopping Plaza,Park,Liquor Store,Coffee Shop,Furniture / Home Store
8,"Brockton, Exhibition Place, Parkdale Village",Performing Arts Venue,Breakfast Spot,Café,Coffee Shop,Yoga Studio,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Convenience Store
9,"CFB Toronto, Downsview East",Airport,Park,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#neighborhoods_venues_sorted['Cluster Labels']
toronto_merged = df_Toronto.copy()


In [35]:

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(axis=0, inplace=True)
#toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype('int')
toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",-79.3606,43.6543,3.0,Coffee Shop,Bakery,Pub,Park,Gym / Fitness Center,Theater,Breakfast Spot,Restaurant,Café,Mexican Restaurant
1,M6A,North York,"Lawrence Heights, Lawrence Manor",-79.4648,43.7185,3.0,Furniture / Home Store,Clothing Store,Women's Store,Boutique,Gift Shop,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant
2,M1B,Scarborough,"Rouge, Malvern",-79.1944,43.8067,2.0,Fast Food Restaurant,Women's Store,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Discount Store
3,M4B,East York,"Woodbine Gardens, Parkview Hill",-79.3099,43.7064,3.0,Pizza Place,Fast Food Restaurant,Athletics & Sports,Gastropub,Intersection,Pet Store,Pharmacy,Breakfast Spot,Bank,Gym / Fitness Center
4,M5B,Downtown Toronto,"Ryerson, Garden District",-79.3789,43.6572,3.0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Fast Food Restaurant,Italian Restaurant,Pizza Place,Plaza,Bookstore


Finally, let's visualize the resulting clusters

In [36]:
toronto_merged = toronto_merged.astype({"Cluster Labels": int})
toronto_merged.dtypes
#print(toronto_merged.shape)

Postcode                  object
Borough                   object
Neighborhood              object
Longitude                 object
Latitude                  object
Cluster Labels             int64
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
4th Most Common Venue     object
5th Most Common Venue     object
6th Most Common Venue     object
7th Most Common Venue     object
8th Most Common Venue     object
9th Most Common Venue     object
10th Most Common Venue    object
dtype: object

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

<a id="item3.4"></a>
### 3.4. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,0,Bar,Women's Store,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar


#### Cluster 2

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,1,Cafeteria,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Discount Store


#### Cluster 3

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,2,Fast Food Restaurant,Women's Store,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Discount Store


#### Cluster 4

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3,Coffee Shop,Bakery,Pub,Park,Gym / Fitness Center,Theater,Breakfast Spot,Restaurant,Café,Mexican Restaurant
1,North York,3,Furniture / Home Store,Clothing Store,Women's Store,Boutique,Gift Shop,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant
3,East York,3,Pizza Place,Fast Food Restaurant,Athletics & Sports,Gastropub,Intersection,Pet Store,Pharmacy,Breakfast Spot,Bank,Gym / Fitness Center
4,Downtown Toronto,3,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Fast Food Restaurant,Italian Restaurant,Pizza Place,Plaza,Bookstore
5,Etobicoke,3,Golf Course,Bank,Women's Store,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar
7,North York,3,Gym,Coffee Shop,Asian Restaurant,Beer Store,Clothing Store,Bike Shop,Dim Sum Restaurant,Japanese Restaurant,Italian Restaurant,Shopping Mall
8,Etobicoke,3,Pharmacy,Beer Store,Pizza Place,Convenience Store,Café,Shopping Plaza,Park,Liquor Store,Coffee Shop,Furniture / Home Store
9,Scarborough,3,Medical Center,Electronics Store,Pizza Place,Intersection,Spa,Breakfast Spot,Rental Car Location,Mexican Restaurant,Dim Sum Restaurant,Dessert Shop
10,North York,3,Coffee Shop,Middle Eastern Restaurant,Sushi Restaurant,Diner,Bridal Shop,Sandwich Place,Deli / Bodega,Fast Food Restaurant,Bank,Restaurant
11,Downtown Toronto,3,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Bar,Asian Restaurant,Bakery,Hotel,Gym


#### Cluster 5

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,North York,4,Baseball Field,Furniture / Home Store,Women's Store,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar


### Thank you reviewing my Assignment!

This notebook was modified by Carlos Pinillos from the one created by [Alex Aklson](https://www.linkedin.com/in/aklson/) and [Polong Lin](https://www.linkedin.com/in/polonglin/) which is part of a course on Coursera called Applied Data Science Capstone. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2019 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). The original notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).